# Medicare PQRS Scoring Captone Project

Completed June 2019 as part of The Data Incubator's Fellowship program


In [1]:
import re
import mpu
from datetime import datetime
from utils import TextColumnSelectTransformer, load_data, pickle_all, unpickle_all
from pathlib import Path
import numpy as np
import pandas as pd
import graphviz
import scipy
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, LinearRegression, Lasso
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer 
from sklearn import model_selection
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.model_selection import GridSearchCV, GroupKFold, cross_val_score, ShuffleSplit
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import SGDClassifier, HuberRegressor, SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, Normalizer, QuantileTransformer, PowerTransformer, PolynomialFeatures, KBinsDiscretizer
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn import base
from collections import defaultdict
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import featuretools as ft
# Standard plotly imports

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
# import cufflinks
# cufflinks.go_offline(connected=True)
# init_notebook_mode(connected=True)


# params = {'legend.fontsize': 20,
#           'figure.figsize': (15, 10),
#          'axes.labelsize': 20,
#          'axes.titlesize': 20,
#          'xtick.labelsize': 20,
#          'ytick.labelsize': 20}
# pylab.rcParams.update(params)
# plt.close('all')

## Files are stored in the '2015' and '2016' subdirectories, relative to the working directory
#data_path = Path('../jnscott_tdi_data')
dp = np.linspace(0,100,11)
dy = np.linspace(1945,2020,15)
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 200)

Possible ndf16 variables of interest include:

- Numerical: grd_yr
- Boolean/Binary: gndr, assgn, pqrs, ehr, mhi
- Categorical: 

 - Single category per provider: cred, med_sch, pri_spec
 - Multiple categories possible: hosp_afl_1,...,hosp_afl_5, sec_spec_1,..., sec_spec_4, org_pac_id, org_pac_id
 
Possible iep16 variables of interest include:

- Numerical: patient_count, prf_rate (target)
- Boolean/Binary: live_site_ind
- Categorical: collection_type


In [2]:
# ndf16, iep16, census_regions, hosps_data, orgs_data, prov_orgs, prov_hosps, \
# prov_sec_specs, hc_star, hc_ans_pct, hc_lin_mean, vocab_dict = load_data()

Step 1/30, import takes ~1  minutes, start time is  2019-06-05 00:54:04.967333
Done, Finish time is  2019-06-05 01:01:43.619656


In [3]:
# pickle_all(ndf16, iep16, census_regions, hosps_data, orgs_data, prov_orgs, \
#            prov_hosps, prov_sec_specs, hc_star, hc_ans_pct, hc_lin_mean, vocab_dict)

df's and vocab_dict saved to disk.


In [ ]:
ndf16, iep16, census_regions, hosps_data, orgs_data, prov_orgs, prov_hosps, \
prov_sec_specs, hc_star, hc_ans_pct, hc_lin_mean, vocab_dict = unpickle_all()

In [4]:
orgs = (pd.DataFrame(orgs_data.org_pac_id.unique())).rename(index=str, columns={0:'org_pac_id'}).drop_duplicates().dropna()

In [5]:
orgs_cols = orgs.columns.tolist()
orgs_vartypes = [ft.variable_types.Index]
orgs_vardict = dict(zip(orgs_cols, orgs_vartypes))

census_regions_cols = census_regions.columns.tolist()
census_regions_vartypes = [ft.variable_types.Categorical,
                 ft.variable_types.Index,
                 ft.variable_types.Categorical,
                 ft.variable_types.Categorical]
census_regions_vardict = dict(zip(census_regions_cols, census_regions_vartypes))

hosps_data_cols = hosps_data.columns.tolist()
hosps_data_vartypes = [ft.variable_types.Index,
                 ft.variable_types.Categorical,
                 ft.variable_types.ZIPCode]
hosps_data_vardict = dict(zip(hosps_data_cols, hosps_data_vartypes))

hc_lin_mean_cols = hc_lin_mean.columns.tolist()
hc_lin_mean_vartypes = [ft.variable_types.Id,
                 ft.variable_types.Categorical,
                 ft.variable_types.ZIPCode,
                 ft.variable_types.Categorical,
                 ft.variable_types.Numeric,
                 ft.variable_types.Numeric,
                 ft.variable_types.Numeric]
hc_lin_mean_vardict = dict(zip(hc_lin_mean_cols, hc_lin_mean_vartypes))

hc_ans_pct_cols = hc_ans_pct.columns.tolist()
hc_ans_pct_vartypes = [ft.variable_types.Id,
                 ft.variable_types.Categorical,
                 ft.variable_types.ZIPCode,
                 ft.variable_types.Categorical,
                 ft.variable_types.Numeric,
                 ft.variable_types.Numeric,
                 ft.variable_types.Numeric]
hc_ans_pct_vardict = dict(zip(hc_ans_pct_cols, hc_ans_pct_vartypes))

hc_star_cols = hc_star.columns.tolist()
hc_star_vartypes = [ft.variable_types.Id,
                 ft.variable_types.Categorical,
                 ft.variable_types.ZIPCode,
                 ft.variable_types.Categorical,
                 ft.variable_types.Ordinal,
                 ft.variable_types.Numeric,
                 ft.variable_types.Numeric]
hc_star_vardict = dict(zip(hc_star_cols, hc_star_vartypes))

prov_orgs_cols = prov_orgs.columns.tolist()
prov_orgs_vartypes = [ft.variable_types.Id,
                 ft.variable_types.Id]
prov_orgs_vardict = dict(zip(prov_orgs_cols, prov_orgs_vartypes))

prov_hosps_cols = prov_hosps.columns.tolist()
prov_hosps_vartypes = [ft.variable_types.Id,
                 ft.variable_types.Id]
prov_hosps_vardict = dict(zip(prov_hosps_cols, prov_hosps_vartypes))

prov_sec_specs_cols = prov_sec_specs.columns.tolist()
prov_sec_specs_vartypes = [ft.variable_types.Id,
                 ft.variable_types.Categorical]
prov_sec_specs_vardict = dict(zip(prov_sec_specs_cols, prov_sec_specs_vartypes))

orgs_data_cols = orgs_data.columns.tolist()
orgs_data_vartypes = [ft.variable_types.Id,
                 ft.variable_types.Categorical,
                 ft.variable_types.Categorical,
                 ft.variable_types.ZIPCode]
orgs_data_vardict = dict(zip(orgs_data_cols, orgs_data_vartypes))

ndf16_cols = ndf16.columns.tolist()
ndf16_vartypes = [ft.variable_types.Index,
                 ft.variable_types.Categorical,
                 ft.variable_types.Categorical,
                 ft.variable_types.Boolean,
                 ft.variable_types.Boolean,
                 ft.variable_types.Boolean,
                 ft.variable_types.Categorical,
                 ft.variable_types.Categorical,
                 ft.variable_types.Categorical,
                 ft.variable_types.Numeric,
                 ft.variable_types.Text,
                 ft.variable_types.Text,
                 ft.variable_types.Text,
                 ft.variable_types.Text,
                 ft.variable_types.Boolean,
                 ft.variable_types.Boolean,
                 ft.variable_types.Boolean,
                 ft.variable_types.Boolean]
ndf16_vardict = dict(zip(ndf16_cols, ndf16_vartypes))

iep16_cols = iep16.columns.tolist()
iep16_vartypes = [ft.variable_types.Id,
                 ft.variable_types.Categorical,
                 ft.variable_types.Categorical,
                 ft.variable_types.Categorical,
                 ft.variable_types.Categorical,
                 ft.variable_types.Numeric,
                 ft.variable_types.Numeric]
iep16_vardict = dict(zip(iep16_cols, iep16_vartypes))


# Entity Definitions
The "entities" that Feature Tools will use to build features are constructed below from the previously assembled dataframes.

In [6]:
es = ft.EntitySet(id = 'phys_care')
es = es.entity_from_dataframe(entity_id = 'hosps', dataframe = hosps_data, index='hosp_id', \
                              variable_types=hosps_data_vardict, make_index=False, time_index=None, \
                              secondary_time_index=None, already_sorted=False)

In [7]:
es = es.entity_from_dataframe(entity_id = 'orgs', dataframe = orgs, index='org_pac_id', \
                              variable_types=orgs_vardict, make_index=False, time_index=None, \
                              secondary_time_index=None, already_sorted=False)

In [8]:
es = es.entity_from_dataframe(entity_id = 'prov', dataframe = ndf16, index='npi', \
                              variable_types=ndf16_vardict, make_index=False, time_index=None, \
                              secondary_time_index=None, already_sorted=False)

In [9]:
es = es.entity_from_dataframe(entity_id = 'states', dataframe = census_regions, index='st', \
                              variable_types=census_regions_vardict, make_index=False, time_index=None, \
                              secondary_time_index=None, already_sorted=False)

In [10]:
es = es.entity_from_dataframe(entity_id = 'prov_orgs', dataframe = prov_orgs, index='prov_orgs_index', \
                              variable_types=prov_orgs_vardict, make_index=True, time_index=None, \
                              secondary_time_index=None, already_sorted=False)


In [11]:
es = es.entity_from_dataframe(entity_id = 'prov_hosps', dataframe = prov_hosps, index='prov_hosps_index', \
                              variable_types=prov_hosps_vardict, make_index=True, time_index=None, \
                              secondary_time_index=None, already_sorted=False)


In [12]:
es = es.entity_from_dataframe(entity_id = 'prov_sec_specs', dataframe = prov_sec_specs, index='prov_sec_specs_index', \
                              variable_types=prov_sec_specs_vardict, make_index=True, time_index=None, \
                              secondary_time_index=None, already_sorted=False)


In [13]:
es = es.entity_from_dataframe(entity_id = 'orgs_data', dataframe = orgs_data, index='orgs_data_index', \
                              variable_types=orgs_data_vardict, make_index=True, time_index=None, \
                              secondary_time_index=None, already_sorted=False)


In [14]:
es = es.entity_from_dataframe(entity_id = 'score_data', dataframe = iep16, index='iep16_index', \
                              variable_types=iep16_vardict, make_index=True, time_index=None, \
                              secondary_time_index=None, already_sorted=False)


In [15]:
es = es.entity_from_dataframe(entity_id = 'hc_lin_mean', dataframe = hc_lin_mean, \
                              index='hc_lin_mean_index', variable_types=hc_lin_mean_vardict, \
                              make_index=True, time_index=None, secondary_time_index=None, already_sorted=False)


In [16]:
es = es.entity_from_dataframe(entity_id = 'hc_star', dataframe = hc_star, index='hc_star_index', \
                              variable_types=hc_star_vardict, make_index=True, time_index=None, \
                              secondary_time_index=None, already_sorted=False)


In [17]:
es = es.entity_from_dataframe(entity_id = 'hc_ans_pct', dataframe = hc_ans_pct, index='hc_ans_pct_index', \
                              variable_types=hc_ans_pct_vardict, make_index=True, time_index=None, \
                              secondary_time_index=None, already_sorted=False)


# Relationship Definitions
Feature Tools is built on the idea of feature primitives and the stacking of these primitives to generate complex features. The code below defines the relationships between the entities defined above. The EntitySet() object, instatiated as es, is stateful and relationships are added one by one.

In [18]:
r_prov_score = ft.Relationship(es['prov']['npi'],
                     es['score_data']['npi'])
es = es.add_relationship(r_prov_score)

In [19]:
r_prov_prov_hosps = ft.Relationship(es['prov']['npi'],
                     es['prov_hosps']['npi'])
es = es.add_relationship(r_prov_prov_hosps)

In [20]:
r_prov_prov_orgs = ft.Relationship(es['prov']['npi'],
                     es['prov_orgs']['npi'])
es = es.add_relationship(r_prov_prov_orgs)

In [21]:
r_prov_prov_sec_specs = ft.Relationship(es['prov']['npi'],
                     es['prov_sec_specs']['npi'])
es = es.add_relationship(r_prov_prov_sec_specs)

In [22]:
r_orgs_prov_orgs = ft.Relationship(es['orgs']['org_pac_id'],
                     es['prov_orgs']['org_pac_id'])
es = es.add_relationship(r_orgs_prov_orgs)

In [23]:
r_orgs_orgs_data = ft.Relationship(es['orgs']['org_pac_id'],
                     es['orgs_data']['org_pac_id'])
es = es.add_relationship(r_orgs_orgs_data)

In [24]:
r_hosps_prov_hosps = ft.Relationship(es['hosps']['hosp_id'],
                     es['prov_hosps']['hosp_id'])
es = es.add_relationship(r_hosps_prov_hosps)

In [25]:
r_hosps_hc_lin_mean = ft.Relationship(es['hosps']['hosp_id'],
                     es['hc_lin_mean']['hosp_id'])
es = es.add_relationship(r_hosps_hc_lin_mean)

In [26]:
r_states_hosps = ft.Relationship(es['states']['st'],
                     es['hosps']['state'])
es = es.add_relationship(r_states_hosps)

In [27]:
r_states_orgs_data = ft.Relationship(es['states']['st'],
                     es['orgs_data']['st'])
es = es.add_relationship(r_states_orgs_data)

In [28]:
dfs_ignore_dict = {'prov':['npi'], \
                   'score_data':['prf_rate'], 'hosps':['zip_code'], 'states':['state'], \
                   'orgs_data':['zip'], 'prov_hosps':['npi'], 'prov_orgs':['npi'], \
                   'prov_sec_specs':['npi']}

# Feature Engineering Using Feature Tools

The below code generates a non-deterministic error related to how dask stops worker jobs. Results are unaffected and returned normally, and match those generated when n_jobs = 1 and no error is generated.

In [ ]:
max_depth = 6
print('Using max_depth=', max_depth)
print('Start time: ', datetime.now(), '\n')
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                       target_entity='score_data',
                                       max_depth=max_depth,
                                       n_jobs=4,
#                                      features_only=True,
                                     ignore_variables=dfs_ignore_dict,
                                     return_variable_types='all')
print('Start time: ', datetime.now())

With max_depth = 6, 174 novel features were generated using Feature Tools.

In [ ]:
#feature_matrix['prf_rate'] = iep16.prf_rate
#feature_matrix.to_pickle('feature_matrix_6_new.df')
feature_matrix = pd.read_pickle('feature_matrix_6_new.df')

In [29]:
ndf16.head(1)

,npi,gndr,assgn,pqrs,ehr,mhi,cred,med_sch,pri_spec,grd_yr,orgs_text,hosps_text,sec_specs_text,states_text,chiro_school,osteo_school,opto_school,dent_school
0,1003000126,M,T,T,F,F,UNREPORTED,OTHER,INTERNAL MEDICINE,1994.0,6103174255,490126,NONE,VA,0,0,0,0


In [ ]:
len(feature_defs), feature_defs

# The contents of the feature matrix are used to identify measures of interest. Amongst the measures that stand out with a relatively poor score (54) but affecting a large number of patients (27 million) and involving a large number of providers (16000) is PQRS_EP_110_1, "Preventive Care and Screening: Influenza Immun..."

In [ ]:
d = {'npi': ['count'], 'patient_count': ['sum'], 'prf_rate':['mean']}
per_measure_nums = feature_matrix.groupby(['measure_id','measure_title'], as_index=False).agg(d)
per_measure_nums.columns = ['measure_id','measure_title','count_provider','sum_patients','mean_prf_rate']

In [ ]:
per_measure_nums.sort_values('mean_prf_rate')

In [ ]:
feature_matrix.columns = [c.lower().replace('.', '_') for c in feature_matrix.columns]


In [ ]:
pqrs110 = feature_matrix.loc[feature_matrix.measure_id == 'PQRS_EP_110_1'].reset_index(drop=True)
pqrs110 = pqrs110.dropna(axis=1, how='any')

In [ ]:
X = pqrs110.drop(columns='prf_rate')
y = pqrs110['prf_rate']
g = X.columns.to_series().groupby(X.dtypes).groups
float_cols = g[list(g.keys())[1]].tolist()
int_cols = g[list(g.keys())[0]].tolist()[1:]
int_cols + float_cols
X_num = X[float_cols+int_cols]
#y = np.log(pqrs110['prf_rate'])

In [ ]:
X.dtypes

In [30]:
vocab_dict['sec_spec_vocab']

['PREVENTATIVE MEDICINE',
 'COLORECTAL SURGERY (PROCTOLOGY)',
 'UNDERSEA AND HYPERBARIC MEDICINE',
 'HEMATOPOIETIC CELL TRANSPLANTATION AND CELLULAR TH',
 'GASTROENTEROLOGY',
 'ALLERGY/IMMUNOLOGY',
 'PHYSICAL MEDICINE AND REHABILITATION',
 'INTERVENTIONAL PAIN MANAGEMENT',
 'GERIATRIC MEDICINE',
 'GENERAL SURGERY',
 'DIAGNOSTIC RADIOLOGY',
 'NEPHROLOGY',
 'HAND SURGERY',
 'PODIATRY',
 'RHEUMATOLOGY',
 'INTERVENTIONAL RADIOLOGY',
 'OPHTHALMOLOGY',
 'DERMATOLOGY',
 'SURGICAL ONCOLOGY',
 'ADVANCED HEART FAILURE AND TRANSPLANT CARDIOLOGY',
 'CRITICAL CARE (INTENSIVISTS)',
 'INTERVENTIONAL CARDIOLOGY',
 'ANESTHESIOLOGY',
 'NUCLEAR MEDICINE',
 'None',
 'NEUROSURGERY',
 'NEUROPSYCHIATRY',
 'OTOLARYNGOLOGY',
 'PSYCHIATRY',
 'ORTHOPEDIC SURGERY',
 'MEDICAL GENETICS AND GENOMICS',
 'OPTOMETRY',
 'VASCULAR SURGERY',
 'PULMONARY DISEASE',
 'CHIROPRACTIC',
 'GYNECOLOGICAL ONCOLOGY',
 'CARDIAC SURGERY',
 'NONE',
 'FAMILY MEDICINE',
 'ENDOCRINOLOGY',
 'PERIPHERAL VASCULAR DISEASE',
 'HEMATOLOGY',
 'O

In [42]:
# class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):

#     def __init__(self, col_name):
#         self.col_name = col_name  # We will need these in transform()

#     def fit(self, X, y=None):
#         # This transformer doesn't need to learn anything about the data,
#         # so it can just return self without any further processing
#         return self

#     def transform(self, X):
#         return [item for sublist in X[self.col_name].values.tolist() for item in sublist]


# class TextColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):

#     def __init__(self, col_name):
#         self.col_name = col_name  # We will need these in transform()

#     def fit(self, X, y=None):
#         # This transformer doesn't need to learn anything about the data,
#         # so it can just return self without any further processing
#         return self

#     def transform(self, X):
# #        return np.array(X[self.col_name].apply(lambda _string: _string.split(',')).apply(lambda _list: {_string.strip() for _string in _list}))
#         return [X[name].apply(lambda _string: _string.split(',')).apply(lambda _list: np.array(_string.strip() for _string in _list)).values for name in self.col_name][0]
    
#     #    [[d[name] for name in self.col_names] for d in X]

class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self):
        self = self  # We will need these in transform()
    
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        return [row.hosps_text for row in X.itertuples()]
#        return [row.org_pac_ids for row in X.itertuples()]

In [ ]:
org_pac_features2 = ['org_pac_ids']
org_pac_transformer2 = Pipeline(steps=[
    ('cst', ColumnSelectTransformer()),
    ('cntvec', CountVectorizer(vocabulary=org_vocab))
    ])

hosps_features2 = ['org_pac_ids']
hosps_transformer2 = Pipeline(steps=[
    ('cst', ColumnSelectTransformer()),
    ('cntvec', CountVectorizer(vocabulary=hosps_vocab))
    ])

preprocessor2 = ColumnTransformer(
    transformers=[
#Put shuffle in somewhere somehow!
        ('num', numeric_transformer2, numeric_features2),
        ('cat', categorical_transformer2, categorical_features2),
        ('org_pac_txt', org_pac_transformer2, org_pac_features2),
        ('hosps_txt', hosps_transformer2, hosps_features2)
    ])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf2 = Pipeline(steps=[('preprocessor', preprocessor2),
                      ('mlp', MLPRegressor(max_iter=100, warm_start=False))
#                      ('rdg', Ridge(alpha=[1.0]))
                      ])

In [45]:
# numeric_features = float_cols + int_cols
# numeric_transformer = Pipeline(steps=[
#     ('norm', Normalizer()),
#     ])

sec_specs_features = ['hosps_text']
sec_specs_transformer = Pipeline(steps=[
    ('cst', ColumnSelectTransformer()),
    ('cntvec', CountVectorizer(vocabulary=vocab_dict['hosps_id_vocab']))

#    ('mlb', MultiLabelBinarizer(classes=vocab_dict['sec_spec_vocab']))
    ])

# pri_spec_features = ['pri_spec']
# pri_spec_transformer = Pipeline(steps=[
#     ('onehot', MultiLabelBinarizer(classes=[pri_spec_cats]))])

categorical_features = ['collection_type', 'live_site_ind', 'assgn', 'ehr', 'mhi', 'gndr', 'st', 'region', 'division']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())])

# org_pac_features = ['org_pac_ids']
# org_pac_transformer = Pipeline(steps=[
#     ('cst', ColumnSelectTransformer(['org_pac_ids'])),
#     ('cntvec', CountVectorizer(vocabulary=org_vocab))
#     ])

# hosps_features = ['hosps']
# hosps_transformer = Pipeline(steps=[
#     ('cst', ColumnSelectTransformer(['hosps'])),
#     ('cntvec', CountVectorizer(vocabulary=hosps_vocab))
#     ])

preprocessor = ColumnTransformer(
    transformers=[
#        ('nmt', numeric_transformer, numeric_features),
         ('spt', sec_specs_transformer, sec_specs_features)
#         ('med', med_sch_transformer, med_sch_features),
#         ('meas', measure_id_transformer, measure_id_features),        
#         ('cat', categorical_transformer, categorical_features),
#         ('org', org_pac_transformer, org_pac_features),
#         ('hosps', hosps_transformer, hosps_features)
    ])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
prep = Pipeline(steps=[('preprocessor', preprocessor)
#                      ('tsvd', TruncatedSVD(n_components=1000))
#                      ('mlp', MLPRegressor(max_iter=60))
#                      ('rfr', RandomForestRegressor(n_estimators=50))
#                      ('lnr', LinearRegression())
#                      ('rdg', Ridge())
                     ])

In [46]:
a=prep.fit_transform(ndf16)

#type(a[0][0])
a
#X[numeric_features]

<1069803x4739 sparse matrix of type '<class 'numpy.int64'>'
	with 1097383 stored elements in Compressed Sparse Row format>

In [44]:
#ndf16.hosps_text
vocab_dict['hosps_id_vocab']

['050112',
 '141350',
 '271333',
 '040020',
 '390160',
 '380014',
 '381324',
 '230264',
 '271327',
 '180049',
 '240001',
 '140280',
 '531311',
 '070019',
 '190004',
 '460004',
 '230047',
 '010110',
 '281357',
 '190255',
 '500058',
 '050131',
 '190041',
 '400113',
 '450831',
 '450399',
 '171369',
 '090004',
 '210027',
 '280032',
 '410004',
 '151320',
 '111327',
 '501325',
 '261323',
 '420007',
 '141341',
 '021311',
 '371323',
 '100084',
 '320085',
 '100114',
 '090008',
 '520189',
 '030122',
 '390112',
 '511309',
 '271312',
 '050069',
 '050776',
 '260110',
 '121300',
 '330223',
 '261336',
 '441311',
 '370083',
 '140062',
 '460058',
 '361333',
 '320005',
 '261325',
 '281363',
 '241325',
 '110234',
 '450039',
 '420098',
 '670093',
 '500030',
 '210032',
 '450079',
 '110074',
 '431339',
 '360137',
 '340097',
 '110071',
 '100071',
 '400018',
 '340073',
 '281304',
 '140046',
 '420083',
 '010149',
 '140011',
 '370019',
 '371317',
 '450883',
 '261310',
 '161378',
 '440161',
 '060030',
 '400120',

In [ ]:
# Compare Algorithms
import matplotlib.pyplot as plt
# prepare configuration for cross validation test harness
seed = 42
# prepare models
models = []
models.append(('mlp', MLPRegressor(max_iter=100)))
models.append(('rfr', RandomForestRegressor(n_estimators=100)))
models.append(('lr', LinearRegression()))
models.append(('rdg', Ridge()))
models.append(('lasso', Lasso()))
# evaluate each model in turn
results = []
names = []
scoring = 'r2'
for name, model in models:
    kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_num, y, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
# Compare Algorithms
import matplotlib.pyplot as plt
# prepare configuration for cross validation test harness
seed = 42
# prepare models
models = []
models.append(('mlp', MLPRegressor(max_iter=100)))
models.append(('rfr', RandomForestRegressor(n_estimators=100)))
models.append(('lr', LinearRegression()))
models.append(('rdg', Ridge()))
models.append(('lasso', Lasso()))
# evaluate each model in turn
results = []
names = []
scoring = 'r2'
for name, model in models:
    kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_num, y, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
rfr = RandomForestRegressor(n_estimators=300)
rfr.fit(X_num,y)


In [ ]:
rfr.score(X_num,y)

In [ ]:
sorted(list(zip(rfr.feature_importances_.tolist(), X_num.columns.tolist())))[::-1]

In [ ]:
# Compare Algorithms
import matplotlib.pyplot as plt
# prepare configuration for cross validation test harness
seed = 42
# prepare models
models = []
#models.append(('mlp', MLPRegressor(max_iter=20)))
models.append(('rfr', RandomForestRegressor(n_estimators=20)))
models.append(('lr', LinearRegression()))
models.append(('rdg', Ridge()))
models.append(('lasso', Lasso()))
# evaluate each model in turn
results = []
names = []
scoring = 'r2'
for name, model in models:
    kfold = model_selection.KFold(n_splits=3, shuffle=True, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_num, y, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
#X, y = [[thing] for thing in mdf_fnl.prac_yrs], [[thing] for thing in mdf_fnl.prf_rate]
cv = model_selection.ShuffleSplit(n_splits=20, test_size=0.2, random_state=42)
param_grid = {"rfr__n_estimators": [400, 500]}    # parameters to Pipeline take the form [label]__[estimator_param]

rfr_pipe = Pipeline([('rfr', RandomForestRegressor())
                                     ])

rfr_cv = model_selection.GridSearchCV(rfr_pipe, 
                                                param_grid=param_grid, 
                                                cv=cv, 
                                                scoring='r2',
                                                refit=True,
                                                n_jobs=-1)
rfr_cv.fit(X_num, y)

#param_grid = {"rfr__max_depth": (1, 2, 3, 4, 5, 6, 7)}    # parameters to Pipeline take the form [label]__[estimator_param]

#td_rfr = Pipeline([('rfr', ('rfr', RandomForestRegressor(max_depth=7, random_state=0, n_estimators=100)))
#                                     ])

#td_rfr_cv = model_selection.GridSearchCV(td_rfr, 
#                                                       param_grid=param_grid, cv=cv, 
#                                                       scoring='r2',
#                                                        refit=True)
#td_rfr_cv.fit(X, y)

In [ ]:
rfr_cv.best_score_

In [ ]:
sorted(list(zip(rfr_cv.feature_importances_.tolist(), X_num.columns.tolist())))[::-1]

In [ ]:
# build a classifier
rgr = Ridge()

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(rfr, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)